In [ ]:
"""
CAPSTONE PROJECT: MULTI-CLASS MENTAL HEALTH CLASSIFICATION
Complete notebook with preprocessing, data loading, and all 6 transformer models
"""

# ===============================================================================
# CELL 0: INSTALL REQUIRED PACKAGES
# ===============================================================================

import subprocess
import sys

def install_packages():
    """Install required packages with compatible versions for Kaggle"""
    
    packages_to_install = [
        ("numpy", "numpy<2.0"),
        ("scikit-learn", "scikit-learn>=1.0.0"),
        ("pandas", "pandas"),
        ("matplotlib", "matplotlib"),
        ("seaborn", "seaborn"),
        ("nltk", "nltk"),
        ("torch", "torch"),
        ("transformers", "transformers"),
        ("datasets", "datasets")
    ]
    
    installed_count = 0
    for package_name, package_spec in packages_to_install:
        try:
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", "-q", package_spec],
                stderr=subprocess.DEVNULL
            )
            print(f"✓ {package_name}")
            installed_count += 1
        except Exception as e:
            print(f"⚠ {package_name} - {str(e)[:50]}")
    
    print(f"\n✓ Installation complete! {installed_count}/{len(packages_to_install)} packages ready.")
    print("⚠ Note: Some Kaggle pre-existing conflicts are normal and safe to ignore.\n")

install_packages()

In [ ]:
# # ===============================================================================
# # FIX CUDNN COMPATIBILITY (KAGGLE P100)
# # ===============================================================================

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# # Disable cuDNN to avoid version mismatch on Kaggle
# tf.config.list_physical_devices('GPU')
# tf.config.set_visible_devices([], 'GPU')

# # Re-enable GPU with memory growth to prevent OOM
# gpus = tf.config.list_physical_devices('GPU')
# for gpu in gpus:
#     try:
#         tf.config.experimental.set_memory_growth(gpu, True)
#     except RuntimeError as e:
#         print(f"GPU setup warning (safe to ignore): {e}")

# print("✓ GPU configured and cuDNN compatibility fixed!")

In [ ]:
# ===============================================================================
# CELL 1: CLEAN IMPORTS & SETUP
# ===============================================================================
import os
import re
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn

# Hugging Face & Sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    pipeline
)

# Settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

# Random Seed (For Reproducibility)
SEED = 42
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(SEED)

print(f"✓ Libraries imported. Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

In [ ]:
# 2. CONFIGURATION
DATA_PATH = "/kaggle/input/mental-health/Combined Data.csv"
OUT_DIR = "distilbert-mental-health"
BATCH_SIZE = 16          # DistilBERT is small, so 16 usually fits on P100 GPU
EPOCHS = 3
LR = 2e-5                # Standard learning rate for Transformers
SEED = 42

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(SEED)


# Load data from Kaggle dataset 
filepath = "/kaggle/input/mental-health/Combined Data.csv"
df = pd.read_csv(filepath)

# Standardize columns
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])
df = df.rename(columns={"statement": "text", "status": "label_name"})

# Drop Nulls/Duplicates
df = df.dropna(subset=["text", "label_name"]).drop_duplicates(subset=["text"]).reset_index(drop=True)

# 🔹 Filter to 5 target classes (drop Normal + Personality disorder)
keep_labels = ["Anxiety", "Depression", "Suicidal", "Stress", "Bipolar"]
df = df[df["label_name"].isin(keep_labels)].reset_index(drop=True)

print("Updated class distribution:")
print(df["label_name"].value_counts())

# Encode Labels (String -> Integer) AFTER filtering
label_values = sorted(df["label_name"].unique())
label2id = {name: i for i, name in enumerate(label_values)}
id2label = {i: name for name, i in label2id.items()}
df["label"] = df["label_name"].map(label2id)

print(f"✓ Data Loaded. Shape: {df.shape}")
print(f"✓ Classes: {label2id}")


# Minimal Cleaning Function (Best for Transformers)
def minimal_clean(text):
    text = str(text)
    text = re.sub(r"http\S+|www\S+|@\w+", "", text)  # Remove URLs/Mentions
    text = re.sub(r"<.*?>", "", text)                 # Remove HTML
    text = re.sub(r"\s+", " ", text).strip()          # Remove double spaces
    return text

df["text"] = df["text"].apply(minimal_clean)

# 4. SPLIT & TOKENIZE
# Stratified split ensures equal distribution of classes in Train/Test
train_df, test_df = train_test_split(df, test_size=0.15, stratify=df["label"], random_state=SEED)

hf_train = Dataset.from_pandas(train_df[["text", "label"]].reset_index(drop=True))
hf_test = Dataset.from_pandas(test_df[["text", "label"]].reset_index(drop=True))
dataset = DatasetDict({"train": hf_train, "test": hf_test})

# 5. COMPUTE CLASS WEIGHTS (Handle Imbalance)
labels = train_df["label"].values
class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to("cuda" if torch.cuda.is_available() else "cpu")

print(f"⚖️ Class Weights calculated: {class_weights}")

# 6. CUSTOM TRAINER (Fixed for newer Transformers versions)
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # We add **kwargs above to catch 'num_items_in_batch' or any other new args
        
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        # Calculate loss with our custom class weights
        loss_fct = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="macro")
    return {"accuracy": acc, "f1_macro": f1}

results_list = []
print("✅ Scoreboard initialized.")

# DistilBERT

In [ ]:
FRIENDLY_NAME = "DistilBERT"
MODEL_PATH    = "distilbert-base-uncased"

print(f"\n\n{'='*40}")
print(f"🥊 TRAINING ROUND: {FRIENDLY_NAME}")
print(f"{'='*40}")

# 1. Tokenize
print(f">>> Tokenizing for {FRIENDLY_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=False, max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 2. Load Model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# 3. STAGE 1: WARMUP (Freeze Body)
print(f"❄️ Freezing backbone for warmup...")
for param in model.base_model.parameters():
    param.requires_grad = False
    
warmup_args = TrainingArguments(
    output_dir=f"comparison_{FRIENDLY_NAME}_warmup",
    learning_rate=1e-3,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

Trainer(
    model=model, args=warmup_args, 
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator
).train()

# 4. STAGE 2: FINE-TUNE (Unfreeze All)
print(f"🔥 Unfreezing for full training...")
for param in model.base_model.parameters():
    param.requires_grad = True
    
training_args = TrainingArguments(
    output_dir=f"comparison_{FRIENDLY_NAME}_final",
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# 5. EVALUATION
print(f"\n📊 {FRIENDLY_NAME} Results:")

# Get Predictions
preds_output = trainer.predict(tokenized_datasets["test"])
y_preds = np.argmax(preds_output.predictions, axis=1)
y_true = preds_output.label_ids

# A. Print Classification Report
print(classification_report(y_true, y_preds, target_names=label_values))

# B. Plot Confusion Matrix
fig, ax = plt.subplots(figsize=(10, 8))
cm = confusion_matrix(y_true, y_preds, labels=range(len(label_values)))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_values)
disp.plot(cmap='Blues', ax=ax, xticks_rotation=45)
plt.title(f"Confusion Matrix: {FRIENDLY_NAME}")
plt.grid(False)
plt.show()

# C. Save Score (Only Once!)
metrics = trainer.evaluate()
results_list.append({
    "Model": FRIENDLY_NAME,
    "Accuracy": metrics["eval_accuracy"],
    "F1_Macro": metrics["eval_f1_macro"]
})

# Cleanup
del model, trainer, tokenizer
torch.cuda.empty_cache()
print(f"✅ {FRIENDLY_NAME} Finished & Cleared from Memory.")

# RoBERTa

In [ ]:
FRIENDLY_NAME = "RoBERTa"
MODEL_PATH    = "roberta-base"

print(f"\n\n{'='*40}")
print(f"🥊 TRAINING ROUND: {FRIENDLY_NAME}")
print(f"{'='*40}")

# 1. Tokenize
print(f">>> Tokenizing for {FRIENDLY_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=False, max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 2. Load Model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# 3. STAGE 1: WARMUP (Freeze Body)
print(f"❄️ Freezing backbone for warmup...")
for param in model.base_model.parameters():
    param.requires_grad = False
    
warmup_args = TrainingArguments(
    output_dir=f"comparison_{FRIENDLY_NAME}_warmup",
    learning_rate=1e-3,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

Trainer(
    model=model, args=warmup_args, 
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator
).train()

# 4. STAGE 2: FINE-TUNE (Unfreeze All)
print(f"🔥 Unfreezing for full training...")
for param in model.base_model.parameters():
    param.requires_grad = True
    
training_args = TrainingArguments(
    output_dir=f"comparison_{FRIENDLY_NAME}_final",
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# 5. EVALUATION
print(f"\n📊 {FRIENDLY_NAME} Results:")

# Get Predictions
preds_output = trainer.predict(tokenized_datasets["test"])
y_preds = np.argmax(preds_output.predictions, axis=1)
y_true = preds_output.label_ids

# A. Print Classification Report
print(classification_report(y_true, y_preds, target_names=label_values))

# B. Plot Confusion Matrix
fig, ax = plt.subplots(figsize=(10, 8))
cm = confusion_matrix(y_true, y_preds, labels=range(len(label_values)))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_values)
disp.plot(cmap='Blues', ax=ax, xticks_rotation=45)
plt.title(f"Confusion Matrix: {FRIENDLY_NAME}")
plt.grid(False)
plt.show()

# C. Save Score (Only Once!)
metrics = trainer.evaluate()
results_list.append({
    "Model": FRIENDLY_NAME,
    "Accuracy": metrics["eval_accuracy"],
    "F1_Macro": metrics["eval_f1_macro"]
})

# Cleanup
del model, trainer, tokenizer
torch.cuda.empty_cache()
print(f"✅ {FRIENDLY_NAME} Finished & Cleared from Memory.")

# BioBERT

In [ ]:
FRIENDLY_NAME = "BioBERT"
MODEL_PATH    = "dmis-lab/biobert-v1.1"

print(f"\n\n{'='*40}")
print(f"🥊 TRAINING ROUND: {FRIENDLY_NAME}")
print(f"{'='*40}")

# 1. Tokenize
print(f">>> Tokenizing for {FRIENDLY_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=False, max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 2. Load Model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# 3. STAGE 1: WARMUP (Freeze Body)
print(f"❄️ Freezing backbone for warmup...")
for param in model.base_model.parameters():
    param.requires_grad = False
    
warmup_args = TrainingArguments(
    output_dir=f"comparison_{FRIENDLY_NAME}_warmup",
    learning_rate=1e-3,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

Trainer(
    model=model, args=warmup_args, 
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator
).train()

# 4. STAGE 2: FINE-TUNE (Unfreeze All)
print(f"🔥 Unfreezing for full training...")
for param in model.base_model.parameters():
    param.requires_grad = True
    
training_args = TrainingArguments(
    output_dir=f"comparison_{FRIENDLY_NAME}_final",
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# 5. EVALUATION
print(f"\n📊 {FRIENDLY_NAME} Results:")

# Get Predictions
preds_output = trainer.predict(tokenized_datasets["test"])
y_preds = np.argmax(preds_output.predictions, axis=1)
y_true = preds_output.label_ids

# A. Print Classification Report
print(classification_report(y_true, y_preds, target_names=label_values))

# B. Plot Confusion Matrix
fig, ax = plt.subplots(figsize=(10, 8))
cm = confusion_matrix(y_true, y_preds, labels=range(len(label_values)))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_values)
disp.plot(cmap='Blues', ax=ax, xticks_rotation=45)
plt.title(f"Confusion Matrix: {FRIENDLY_NAME}")
plt.grid(False)
plt.show()

# C. Save Score (Only Once!)
metrics = trainer.evaluate()
results_list.append({
    "Model": FRIENDLY_NAME,
    "Accuracy": metrics["eval_accuracy"],
    "F1_Macro": metrics["eval_f1_macro"]
})

# Cleanup
del model, trainer, tokenizer
torch.cuda.empty_cache()
print(f"✅ {FRIENDLY_NAME} Finished & Cleared from Memory.")

# PubMedBERT

In [ ]:
FRIENDLY_NAME = "PubMedBERT"
MODEL_PATH    = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"

print(f"\n\n{'='*40}")
print(f"🥊 TRAINING ROUND: {FRIENDLY_NAME}")
print(f"{'='*40}")

# 1. Tokenize
print(f">>> Tokenizing for {FRIENDLY_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=False, max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 2. Load Model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# 3. STAGE 1: WARMUP (Freeze Body)
print(f"❄️ Freezing backbone for warmup...")
for param in model.base_model.parameters():
    param.requires_grad = False
    
warmup_args = TrainingArguments(
    output_dir=f"comparison_{FRIENDLY_NAME}_warmup",
    learning_rate=1e-3,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

Trainer(
    model=model, args=warmup_args, 
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator
).train()

# 4. STAGE 2: FINE-TUNE (Unfreeze All)
print(f"🔥 Unfreezing for full training...")
for param in model.base_model.parameters():
    param.requires_grad = True
    
training_args = TrainingArguments(
    output_dir=f"comparison_{FRIENDLY_NAME}_final",
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# 5. EVALUATION
print(f"\n📊 {FRIENDLY_NAME} Results:")

# Get Predictions
preds_output = trainer.predict(tokenized_datasets["test"])
y_preds = np.argmax(preds_output.predictions, axis=1)
y_true = preds_output.label_ids

# A. Print Classification Report
print(classification_report(y_true, y_preds, target_names=label_values))

# B. Plot Confusion Matrix
fig, ax = plt.subplots(figsize=(10, 8))
cm = confusion_matrix(y_true, y_preds, labels=range(len(label_values)))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_values)
disp.plot(cmap='Blues', ax=ax, xticks_rotation=45)
plt.title(f"Confusion Matrix: {FRIENDLY_NAME}")
plt.grid(False)
plt.show()

# C. Save Score (Only Once!)
metrics = trainer.evaluate()
results_list.append({
    "Model": FRIENDLY_NAME,
    "Accuracy": metrics["eval_accuracy"],
    "F1_Macro": metrics["eval_f1_macro"]
})

# Cleanup
del model, trainer, tokenizer
torch.cuda.empty_cache()
print(f"✅ {FRIENDLY_NAME} Finished & Cleared from Memory.")

# ClinicalBERT

In [ ]:
FRIENDLY_NAME = "ClinicalBERT"
MODEL_PATH    = "emilyalsentzer/Bio_ClinicalBERT"

print(f"\n\n{'='*40}")
print(f"🥊 TRAINING ROUND: {FRIENDLY_NAME}")
print(f"{'='*40}")

# 1. Tokenize
print(f">>> Tokenizing for {FRIENDLY_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=False, max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 2. Load Model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# 3. STAGE 1: WARMUP (Freeze Body)
print(f"❄️ Freezing backbone for warmup...")
for param in model.base_model.parameters():
    param.requires_grad = False
    
warmup_args = TrainingArguments(
    output_dir=f"comparison_{FRIENDLY_NAME}_warmup",
    learning_rate=1e-3,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

Trainer(
    model=model, args=warmup_args, 
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator
).train()

# 4. STAGE 2: FINE-TUNE (Unfreeze All)
print(f"🔥 Unfreezing for full training...")
for param in model.base_model.parameters():
    param.requires_grad = True
    
training_args = TrainingArguments(
    output_dir=f"comparison_{FRIENDLY_NAME}_final",
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# 5. EVALUATION
print(f"\n📊 {FRIENDLY_NAME} Results:")

# Get Predictions
preds_output = trainer.predict(tokenized_datasets["test"])
y_preds = np.argmax(preds_output.predictions, axis=1)
y_true = preds_output.label_ids

# A. Print Classification Report
print(classification_report(y_true, y_preds, target_names=label_values))

# B. Plot Confusion Matrix
fig, ax = plt.subplots(figsize=(10, 8))
cm = confusion_matrix(y_true, y_preds, labels=range(len(label_values)))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_values)
disp.plot(cmap='Blues', ax=ax, xticks_rotation=45)
plt.title(f"Confusion Matrix: {FRIENDLY_NAME}")
plt.grid(False)
plt.show()

# C. Save Score (Only Once!)
metrics = trainer.evaluate()
results_list.append({
    "Model": FRIENDLY_NAME,
    "Accuracy": metrics["eval_accuracy"],
    "F1_Macro": metrics["eval_f1_macro"]
})

# Cleanup
del model, trainer, tokenizer
torch.cuda.empty_cache()
print(f"✅ {FRIENDLY_NAME} Finished & Cleared from Memory.")

In [ ]:
# Visualize the Battle
print("\n📊 FINAL RESULTS TABLE:")
df_results = pd.DataFrame(results_list)
print(df_results)

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(data=df_results, x="Model", y="F1_Macro", palette="viridis")
plt.title("Transformer Model Comparison (Macro F1 Score)")
plt.ylabel("F1 Score")
plt.ylim(0.7, 1.0) 

# Add labels
for index, row in df_results.iterrows():
    plt.text(index, row.F1_Macro, f"{row.F1_Macro:.4f}", color='black', ha="center", va='bottom')

plt.show()

In [ ]:
# 8. EVALUATION & SAVING
# ----------------------
print("\n📊 Final Evaluation:")
preds_output = trainer.predict(tokenized_datasets["test"])
y_preds = np.argmax(preds_output.predictions, axis=1)
y_true = preds_output.label_ids

print(classification_report(y_true, y_preds, target_names=label_values))

# Save Model
print(f">>> Saving model to {OUT_DIR}...")
trainer.save_model(OUT_DIR)
tokenizer.save_pretrained(OUT_DIR)

# 9. INTERACTIVE TEST
# -------------------
print("\n🧪 Quick Test:")
classifier = pipeline("text-classification", model=OUT_DIR, device=0 if torch.cuda.is_available() else -1)
text = "I feel really anxious and I cannot sleep at night."
result = classifier(text)
print(f"Input: {text}")
print(f"Prediction: {result}")